<a href="https://colab.research.google.com/github/Marche1os/Z3-solver/blob/master/strategies_and_tactics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!pip install "z3-solver"
from z3 import *

In [37]:
def describe_goal(g):
    print(f"Количество подцелей: {len(g)}")
    for i, subgoal in enumerate(g):
        print(f"Подцель {i}: {subgoal}")

In [38]:
# Задача 1 разбивка клауз
x, y = Ints('x y')

# Формула: (|x| == |y|) AND (x + y > 10)
# Запишем |x| == |y| как (x == y) OR (x == -y)
f = And(Or(x == y, x == -y), x + y > 10)

g = Goal()
g.add(f)
print("Исходная цель:", g)

# Применяем тактику расщепления дизъюнкций
t = Tactic('split-clause')
# Результат применения тактики к цели
subgoals = t(g)

describe_goal(subgoals)
# задача разбилась на две независимые ветки:
# 1. x == y, 2*x > 10
# 2. x == -y, 0 > 10 (эта ветка очевидно противоречива)

Исходная цель: [Or(x == y, x == -y), x + y > 10]
Количество подцелей: 2
Подцель 0: [x == y, x + y > 10]
Подцель 1: [x == -y, x + y > 10]


In [39]:
# Задача 2. Выбор двух стратегий

# цель 1 нелинейная арифметика
x, y = Ints('x y')
g1 = Goal()
g1.add(x*x + y*y > 10)

# цель 2 булева
p, q, r = Bools('p q r')
g2 = Goal()
g2.add(Implies(And(p, q), r))

# стратегия для нелинейной арифметики
arith = OrElse(TryFor(Tactic('nlsat'), 300),
               Tactic('qe'))

# стратегия для простой логики
bool_strategy = Tactic('sat')

# объединяем в par_or (первый успешный)
strategy = ParOr(arith, bool_strategy)

print("Цель 1:")
print(strategy(g1))

print("Цель 2:")
print(strategy(g2))

Цель 1:
[[x*x + y*y > 10]]
Цель 2:
[[Implies(And(p, q), r)]]


In [40]:
# Задача 3 комбинатор OrElse
x = Int('x')

# Уравнение: x > 0 и x < 2.
f = And(x > 0, x < 2)

g = Goal()
g.add(f)

print(tactic_description('smt'))

# Создаем тактику, которая заведомо слабая для целых чисел, например,
# тактика для разщностной логики
# а если не выйдет - используем полноценный smt.

t = OrElse(Tactic('nra'), Tactic('smt'))

res = t(g)
describe_goal(res)
# Тактика SMT решит это моментально, вернув модель или доказательство

apply a SAT based SMT solver.
Количество подцелей: 1
Подцель 0: []


In [41]:
# Задача 4 - выбор тактики

x, y = Ints('x y')

goal = Goal()
goal.add((x + y) * 3 == 15, x*x > 3)

strategy = OrElse(Tactic('solve-eqs'), Tactic('qfnra'))
result = strategy(goal)

print(result)

[[(x + y)*3 == 15, x*x > 3]]


In [48]:
# Задача 5 - решение системы уравнений:
# [
#    a^2 + b^2 = 25;
#.   a * b = 12
# ]
#
a,b = Reals('a b')
solver = Solver()

solver.add(a*a + b*b == 25)
solver.add(a*b == 12)

print(solver.check())
if solver.check() == sat:
    print("Solution:", solver.model())
else:
    print("No solution found")

sat
Solution: [b = 4, a = 3]
